In [1]:
#pip install spglib --user

In [2]:
import numpy as np
from numpy.linalg import norm

from ase.io import read,write
from ase.visualize import view,ngl
from ase.spacegroup import crystal
from ase.spacegroup import Spacegroup
from ase.data import atomic_numbers, atomic_names
from ase import Atoms

import spglib

import nglview

In [3]:
def ase_to_spgcell(ase_atoms=None, cell=None, inverse=False):
    if not inverse:
        assert ase_atoms is not None
        return (ase_atoms.get_cell(),
                ase_atoms.get_scaled_positions(),
                ase_atoms.get_atomic_numbers())
    else:
        assert cell is not None
        return Atoms(cell=cell[0],
                     scaled_positions=cell[1],
                     numbers=cell[2])

## Lattice parameters, angles and spacegroup

In [4]:
a=4.9780
b=4.9780
c=6.9480
alpha=90
beta=90
gamma=90
thespacegroup=92
print(Spacegroup(thespacegroup))

92    P 41 21 2
  setting 1
  centrosymmetric 0
  primitive vectors
     1.0000000000  0.0000000000  0.0000000000
     0.0000000000  1.0000000000  0.0000000000
     0.0000000000  0.0000000000  1.0000000000
  reciprocal vectors
      1   0   0
      0   1   0
      0   0   1
  1 subtranslations
     0.0000000000  0.0000000000  0.0000000000
  8 symmetry operations (rot+trans)
    1  0  0     0  1  0     0  0  1    0.0000000000  0.0000000000  0.0000000000
   -1  0  0     0 -1  0     0  0  1    0.0000000000  0.0000000000  0.5000000000
    0 -1  0     1  0  0     0  0  1    0.5000000000  0.5000000000  0.2500000000
    0  1  0    -1  0  0     0  0  1    0.5000000000  0.5000000000  0.7500000000
   -1  0  0     0  1  0     0  0 -1    0.5000000000  0.5000000000  0.2500000000
    1  0  0     0 -1  0     0  0 -1    0.5000000000  0.5000000000  0.7500000000
    0  1  0     1  0  0     0  0 -1    0.0000000000  0.0000000000  0.0000000000
    0 -1  0    -1  0  0     0  0 -1    0.0000000000  0.00000000

## there are two inequivalent atoms with crystal coordinates:
## O:     0.23976   0.10324   0.17844
## Si:    0.30004   0.30004   0.00000

In [5]:
sio2=crystal(symbols=['O','Si'],
               basis=[(0.23976 ,  0.10324  , 0.17844),
                      (0.30004 ,  0.30004  , 0.00000)],
               spacegroup=thespacegroup, 
               cellpar=[a, b, c, alpha, beta, gamma])

In [6]:
view(sio2, viewer='ngl')

In [7]:
#v=_

In [8]:
# delete all old components
#while hasattr(v.view, "component_0"):
#    v.view.component_0.clear_representations()
#    cid = v.view.component_0.id
#    v.view.remove_component(cid)

In [9]:
#v.view.add_component(nglview.ASEStructure(sio2), 
#                     default_representation=False)
#v.view.add_ball_and_stick(aspectRatio=2.0, opacity=1.0,component=0)

In [10]:
#v.view.add_unitcell()
#v.view.center()

In [11]:
cell = ase_to_spgcell(ase_atoms=sio2)
lattice, scaled_positions, numbers = spglib.find_primitive(cell, symprec=1e-5)
reduced = ase_to_spgcell(cell=(lattice, scaled_positions, numbers),inverse=True)

In [12]:
sym_info = spglib.get_symmetry_dataset(ase_to_spgcell(ase_atoms=sio2), 
                                       symprec=1e-5, angle_tolerance=-1.0, hall_number=0)

In [13]:
sym_info

{'number': 92,
 'hall_number': 369,
 'international': 'P4_12_12',
 'hall': 'P 4abw 2nw',
 'choice': '',
 'transformation_matrix': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'origin_shift': array([5.55111512e-17, 5.55111512e-17, 0.00000000e+00]),
 'rotations': array([[[ 1,  0,  0],
         [ 0,  1,  0],
         [ 0,  0,  1]],
 
        [[ 0, -1,  0],
         [ 1,  0,  0],
         [ 0,  0,  1]],
 
        [[-1,  0,  0],
         [ 0, -1,  0],
         [ 0,  0,  1]],
 
        [[ 0,  1,  0],
         [-1,  0,  0],
         [ 0,  0,  1]],
 
        [[ 1,  0,  0],
         [ 0, -1,  0],
         [ 0,  0, -1]],
 
        [[ 0, -1,  0],
         [-1,  0,  0],
         [ 0,  0, -1]],
 
        [[-1,  0,  0],
         [ 0,  1,  0],
         [ 0,  0, -1]],
 
        [[ 0,  1,  0],
         [ 1,  0,  0],
         [ 0,  0, -1]]], dtype=int32),
 'translations': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.00000000e-01,  5.00000000e-01,  2.5000

In [14]:
rotations = sym_info['rotations']
translations = sym_info['translations']

In [15]:
new_positions=(np.matmul(rotations[1],sym_info['std_positions'].T).T 
               + translations[1]) %1.0

In [16]:
sio2.set_scaled_positions(new_positions)

In [17]:
view(sio2,viewer='ngl')

In [18]:
#v=_

In [19]:
#hasattr(v.view,"component")